In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/churn-mi/1_Assignment_Online Retail.csv


**Importing Libraries**

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn import svm

**Importing Data**

In [3]:
df=pd.read_csv("../input/churn-mi/1_Assignment_Online Retail.csv",encoding= 'unicode_escape')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,01-12-2010 08:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,01-12-2010 08:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,01-12-2010 08:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,01-12-2010 08:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,01-12-2010 08:26,3.39,17850.0,United Kingdom


**Analyzing the Data**

In [4]:
print(df.isnull().any())
print(df.info())
df.shape

InvoiceNo      False
StockCode      False
Description     True
Quantity       False
InvoiceDate    False
UnitPrice      False
CustomerID      True
Country        False
dtype: bool
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB
None


(541909, 8)

In [5]:
df.drop("Description",axis=1,inplace=True)
df=df.dropna()
df.isnull().any()

InvoiceNo      False
StockCode      False
Quantity       False
InvoiceDate    False
UnitPrice      False
CustomerID     False
Country        False
dtype: bool

In [6]:
df['CustomerID']=df['CustomerID'].astype(int)
df[df['Quantity']<0]=0
df['InvoiceDate']=pd.to_datetime(df['InvoiceDate'])
df=df.sort_values("CustomerID")
df=df.replace(r'^s*$', float('NaN'), regex = True)

**Target variable creation**

In [7]:
date1=pd.Timestamp(2010,12,2)
date2=pd.Timestamp(2011,8,31)
date3=pd.Timestamp(2011,9,1)
date4=pd.Timestamp(2011,12,31)
set1=df[df["InvoiceDate"]>=date1]
set1=set1[set1["InvoiceDate"]<=date2]


set2=df[df["InvoiceDate"]>=date3]
set2=set1[set1["InvoiceDate"]<=date4]

In [8]:
set1ID=set1["CustomerID"].unique()
set2ID=set2["CustomerID"].unique()
churn_list=[]
for i in set1ID:
    if i  in set2ID:
        churn_list.append(i)

df=df.assign(IsChurn=[0 if x in churn_list else 1 for x in df["CustomerID"]])
df=df.drop(['InvoiceNo','InvoiceDate'],axis=1)
df['StockCode'] = df['StockCode'].astype('str').str.extractall('(\d+)').unstack().fillna('').sum(axis=1).astype(int)
df=df.dropna(axis=1)
pure_df=pd.get_dummies(df)
pure_without=pure_df.drop('IsChurn', axis=1)
df.head()

,Quantity,UnitPrice,CustomerID,Country,IsChurn
114536,0,0.0,0,0,1
246754,0,0.0,0,0,1
246755,0,0.0,0,0,1
246756,0,0.0,0,0,1
246771,0,0.0,0,0,1


In [9]:
X_train, X_test, y_train, y_test = train_test_split(pure_without,pure_df['IsChurn'], test_size=0.10)

**ID3 Implementation of Decision Tree**

In [10]:
clf = DecisionTreeClassifier()
clf = clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.9981073175527861


*Yes ID3 can be used*

**ANN**

In [11]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(10, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, y_train, batch_size=32, epochs=8, validation_data=(X_test, y_test))
results = model.evaluate(X_test, y_test, batch_size=128)
print("test accuracy:", results[1])

2022-10-07 18:00:41.153815: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2022-10-07 18:00:41.421530: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/8
11443/11443 [==============================] - 29s 2ms/step - loss: 1.1455 - accuracy: 0.8765 - val_loss: 1.3036 - val_accuracy: 0.9208
Epoch 2/8
11443/11443 [==============================] - 27s 2ms/step - loss: 0.8600 - accuracy: 0.8777 - val_loss: 0.8002 - val_accuracy: 0.9207
Epoch 3/8
11443/11443 [==============================] - 28s 2ms/step - loss: 0.8209 - accuracy: 0.8782 - val_loss: 2.1215 - val_accuracy: 0.9208
Epoch 4/8
11443/11443 [==============================] - 27s 2ms/step - loss: 0.7655 - accuracy: 0.8815 - val_loss: 0.6732 - val_accuracy: 0.9207
Epoch 5/8
11443/11443 [==============================] - 30s 3ms/step - loss: 0.7344 - accuracy: 0.8816 - val_loss: 0.6840 - val_accuracy: 0.9208
Epoch 6/8
11443/11443 [==============================] - 27s 2ms/step - loss: 0.6978 - accuracy: 0.8828 - val_loss: 0.5312 - val_accuracy: 0.9208
Epoch 7/8
11443/11443 [==============================] - 26s 2ms/step - loss: 0.6629 - accuracy: 0.8838 - val_loss: 1.4754 -

**SVM**

In [ ]:
classifier = svm.SVC(kernel='linear', random_state=0)
X_train, X_test, y_train, y_test = train_test_split(pure_without,pure_df['IsChurn'], test_size=0.10)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
cm=confusion_matrix(y_test, y_pred)
class_label = ["Negative", "Positive"]
df = pd.DataFrame(cm, index = class_label, columns = class_label)
print ("Accuracy : ",accuracy_score(y_test, y_pred)*100)
report=classification_report(y_test, y_pred)
print(report) 
sns.set()
sns.heatmap(df, annot = True,fmt="d")
plt.title("Test_Confusion_Matrix")
plt.xlabel("Predicted_Label")
plt.ylabel("Actual_Label")
plt.show()